In [3]:
# -*- coding: utf-8 -*-
# ============================================================
#   농산물 거래데이터 EDA (쥬피터 전용 완성본)
#   - 데이터 이해/정리: dtype, 품목 종류, 파생(연/월/주차/요일/일요일)
#   - 결측치 EDA: 전체/품목별×칼럼별 표 + 품목별 결측 매트릭스 + 연도×칼럼 결측 히트맵
#   - 기초통계: 주간평균(6일 기준: 일요일 제외)
#   - 시계열: 월별 추세(품목 비교), 연도별 일 단위(품목별로 연도 분리)
#   - 이상치(급등): 일간 증감률 IQR 상단(Q3 + 1.5*IQR)
#   - 공급-가격 상관 & 시차 상관(lag 1~7)
#   - (선택) 단가 생성: 금액/반입량
# ============================================================

# ============== 0) 기본 설정 ==============
PATH = "../data/전처리데이터/감자_filled_weekly_item.csv"  # TODO: 여러분의 파일 경로로 교체
SAVE_DIR = None   # 예: "/mnt/data/eda_out" (폴더가 없으면 자동 생성). 저장 안 하려면 None

import os, warnings, io, textwrap
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 한글 폰트 (항상 로드)
import koreanize_matplotlib  # noqa: F401

from datetime import datetime
from IPython.display import display, HTML

# 보기 옵션
pd.set_option("display.max_rows", 150)
pd.set_option("display.max_columns", 150)
pd.set_option("display.width", 1500)

def ensure_dir(d):
    if d is None: 
        return
    os.makedirs(d, exist_ok=True)

ensure_dir(SAVE_DIR)

# ============== 1) 로드 & 기본 전처리 ==============
# 인코딩 유연 로드
df = None
for enc in ["utf-8-sig", "cp949", "euc-kr", "utf-8"]:
    try:
        df = pd.read_csv(PATH, encoding=enc)
        break
    except Exception:
        continue
if df is None:
    raise RuntimeError("CSV 로드 실패: 인코딩을 확인하세요. PATH=" + PATH)

# 컬럼 공백 제거
df.columns = [c.strip() for c in df.columns]

# 날짜/품목 컬럼 추론
date_col_candidates = [c for c in df.columns if "거래일자" in c or "date" in c.lower()]
if len(date_col_candidates) == 0:
    raise ValueError("`거래일자`(또는 date 유사) 컬럼을 찾을 수 없습니다.")
date_col = date_col_candidates[0]

item_col_candidates = [c for c in df.columns if ("품목" in c) or ("품종" in c) or ("품목명" in c)]
item_col = item_col_candidates[0] if len(item_col_candidates) else "품목명"
if item_col not in df.columns:
    df[item_col] = "미상"

# 날짜 파싱
df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

# 수치형 숫자화
def to_numeric_clean(s: pd.Series) -> pd.Series:
    return pd.to_numeric(
        s.astype(str)
         .str.replace(",", "", regex=False)
         .str.replace(" ", "", regex=False)
         .str.replace("%", "", regex=False)
         .str.replace("\u2212", "-", regex=False),  # 유니코드 음수 기호
        errors="coerce"
    )

num_candidates = [
    "반입량","금액",
    "전년 반입량","전년 반입량 증감률(%)",
    "전년 금액","전년 금액 증감률(%)",
    "평년 반입량","평년 반입량 증감률(%)",
    "평년 금액","평년 금액 증감률(%)"
]
for c in num_candidates:
    if c in df.columns:
        df[c] = to_numeric_clean(df[c])

# 파생: 연/월/요일/일요일/주차
df["연도"] = df[date_col].dt.year
df["월"] = df[date_col].dt.month
df["weekday"] = df[date_col].dt.weekday   # 0=월, 6=일
df["is_sunday"] = (df["weekday"] == 6)
df["주차"] = df[date_col].dt.isocalendar().week.astype("Int64")

# (선택) 단가 생성: 금액/반입량
if {"금액","반입량"}.issubset(df.columns):
    df["평균단가"] = np.where(df["반입량"]>0, df["금액"]/df["반입량"], np.nan)

# ============== 2) 데이터 개요 ==============
print("=== 데이터 개요 ===")
print(f"파일: {os.path.basename(PATH)}")
print("크기:", df.shape)
print("기간:", f"{df[date_col].min()} ~ {df[date_col].max()}")
print("dtypes:\n", df.dtypes)
print("품목 수:", df[item_col].nunique())
print("품목 목록(앞 20개):", df[item_col].dropna().unique().tolist()[:20])

# 저장용 간단 개요 txt
overview = io.StringIO()
print("=== 데이터 개요 ===", file=overview)
print(f"파일: {os.path.basename(PATH)}", file=overview)
print(f"크기: {df.shape}", file=overview)
print(f"기간: {df[date_col].min()} ~ {df[date_col].max()}", file=overview)
print("dtypes:\n" + str(df.dtypes), file=overview)
print(f"품목 수: {df[item_col].nunique()}", file=overview)
if SAVE_DIR: 
    with open(os.path.join(SAVE_DIR, "EDA_개요.txt"), "w", encoding="utf-8") as f:
        f.write(overview.getvalue())

# ============== 3) 결측치 EDA ==============
print("\n=== 결측 현황(전체) ===")
na_cnt = df.isna().sum().sort_values(ascending=False)
na_ratio = (df.isna().mean()*100).round(2).sort_values(ascending=False)
na_table = pd.concat([na_cnt.rename("결측개수"), na_ratio.rename("결측비율(%)")], axis=1)
display(na_table)

print("\n=== [품목별 × 칼럼별] 결측률(%) (전치) ===")
item_na_pct = (
    df.groupby(item_col)
      .apply(lambda g: (g.isna().mean()*100).round(2))
      .T
)
display(item_na_pct)

# ---- 3-2) 품목별 결측 매트릭스 (샘플) ----
def show_na_matrix_by_item(df, max_rows_for_na_viz=1500, max_items=6):
    items = df[item_col].dropna().unique().tolist()[:max_items]
    for it in items:
        sub = df[df[item_col]==it].copy()
        if len(sub) == 0: 
            continue
        sub = sub.sample(n=min(len(sub), max_rows_for_na_viz), random_state=42).sort_values(date_col)
        na_matrix = sub.isna().astype(int).values
        plt.figure(figsize=(10, 3))
        plt.imshow(na_matrix, aspect="auto", interpolation="nearest")
        plt.title(f"[결측 패턴] {it} (샘플 최대 {max_rows_for_na_viz}행)")
        plt.xlabel("컬럼 인덱스"); plt.ylabel("행(샘플)")
        plt.tight_layout(); plt.show()

print("\n[시각화] 품목별 결측 매트릭스(샘플)")
show_na_matrix_by_item(df)

# ---- 3-3) 연도×칼럼 결측률 히트맵 ----
key_cols = [c for c in ["반입량","금액","평균단가","전년 반입량","전년 금액","평년 반입량","평년 금액"] if c in df.columns]

def heatmap_year_col_missing(df, max_items=6):
    items = df[item_col].dropna().unique().tolist()[:max_items]
    for it in items:
        sub = df[df[item_col]==it].copy()
        if len(sub)==0 or len(key_cols)==0: 
            continue
        heat = (
            sub.groupby("연도")[key_cols]
               .apply(lambda g: (g.isna().mean()*100).round(2))
        )
        if heat.shape[0]==0:
            continue
        arr = heat.values
        plt.figure(figsize=(max(6, 1.6*len(key_cols)), 0.5*max(6, len(heat))))
        plt.imshow(arr, aspect="auto", interpolation="nearest")
        plt.title(f"[연도×칼럼 결측률(%)] {it}")
        plt.xlabel("칼럼"); plt.ylabel("연도")
        plt.xticks(ticks=np.arange(len(key_cols)), labels=key_cols, rotation=45, ha="right")
        plt.yticks(ticks=np.arange(len(heat.index)), labels=heat.index)
        plt.colorbar(label="결측률(%)")
        plt.tight_layout(); plt.show()

print("\n[시각화] 연도×칼럼 결측률 히트맵")
heatmap_year_col_missing(df)

# ============== 4) 주간 평균(6일 기준: 일요일 제외) ==============
agg_cols = [c for c in ["반입량","금액","평균단가"] if c in df.columns]
df_no_sun = df[~df["is_sunday"]].copy()
weekly = (
    df_no_sun
      .groupby([item_col, "연도", "주차"])[agg_cols]
      .sum()
      .reset_index()
)
for c in [col for col in agg_cols if col != "평균단가"]:
    weekly[f"주간평균_{c}(6일기준)"] = weekly[c] / 6.0
# 평균단가는 합이 아닌 가중평균이 더 자연스러우나, 단순화 위해 합/6으로 표기. 필요시 수정.

print("\n=== 주간 평균(6일 기준) 예시 ===")
display(weekly.head())

weekly_stats = (
    weekly.groupby(item_col)[[c for c in weekly.columns if "주간평균" in c]]
          .agg(["mean","median","std","min","max","count"])
)
print("\n=== 품목별 주간 평균 통계량 ===")
display(weekly_stats)

# ============== 5) 월별 합계 추세 (품목 비교) ==============
monthly = df.groupby([item_col, "연도", "월"])[[c for c in ["반입량","금액"] if c in df.columns]].sum().reset_index()
if len(monthly):
    monthly["연-월"] = pd.to_datetime(monthly["연도"].astype(str) + "-" + monthly["월"].astype(str) + "-01")

if "반입량" in monthly.columns:
    plt.figure(figsize=(10, 4))
    for it in monthly[item_col].unique()[:10]:
        sub = monthly[monthly[item_col]==it]
        plt.plot(sub["연-월"], sub["반입량"], label=str(it))
    plt.title("품목별 월 합계 반입량 추세")
    plt.xlabel("월"); plt.ylabel("반입량(월합)")
    if monthly[item_col].nunique() <= 10: plt.legend()
    plt.tight_layout(); plt.show()

if "금액" in monthly.columns:
    plt.figure(figsize=(10, 4))
    for it in monthly[item_col].unique()[:10]:
        sub = monthly[monthly[item_col]==it]
        plt.plot(sub["연-월"], sub["금액"], label=str(it))
    plt.title("품목별 월 합계 금액 추세")
    plt.xlabel("월"); plt.ylabel("금액(월합)")
    if monthly[item_col].nunique() <= 10: plt.legend()
    plt.tight_layout(); plt.show()

# ============== 6) 연도별 일 단위 시계열 (품목별, 최근 N년) ==============
def plot_daily_by_year(df_item: pd.DataFrame, value_col: str, item_name: str, limit_years=5):
    years = sorted(df_item["연도"].dropna().unique())[-limit_years:]
    for y in years:
        sub = df_item[df_item["연도"]==y].sort_values(date_col)
        if len(sub) == 0:
            continue
        plt.figure(figsize=(10, 3))
        plt.plot(sub[date_col], sub[value_col])
        plt.title(f"{item_name} - {value_col} 일 시계열 ({y}년)")
        plt.xlabel("일자"); plt.ylabel(value_col)
        plt.tight_layout(); plt.show()

items = df[item_col].dropna().unique().tolist()
for it in items[:6]:  # 과도 출력 방지
    dfi = df[df[item_col]==it]
    for vc in [c for c in ["반입량","금액","평균단가"] if c in dfi.columns]:
        plot_daily_by_year(dfi, vc, it)

# ============== 7) 이상치(급등) 탐지: IQR 기준 (일간 증감률) ==============
def iqr_upper_threshold(s: pd.Series):
    x = s.dropna()
    if len(x) < 10:
        return np.nan, np.nan, np.nan
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    return (q3 + 1.5*iqr), q1, q3

rows_outliers = []
for it in items:
    dfi = df[df[item_col]==it].sort_values(date_col).copy()
    cols_to_check = [c for c in ["반입량","금액","평균단가"] if c in dfi.columns]
    for c in cols_to_check:
        dfi[f"{c}_pct"] = dfi[c].pct_change()*100
        up, q1, q3 = iqr_upper_threshold(dfi[f"{c}_pct"])
        if pd.notna(up):
            dfi[f"{c}_급등후보"] = dfi[f"{c}_pct"] >= up
    mask_cols = [col for col in dfi.columns if col.endswith("급등후보")]
    if mask_cols:
        mask = dfi[mask_cols].any(axis=1)
        out_table = dfi.loc[mask, [date_col, item_col] + cols_to_check + [f"{c}_pct" for c in cols_to_check] + mask_cols]
        if len(out_table):
            rows_outliers.append(out_table.tail(50))

outliers_df = pd.concat(rows_outliers) if rows_outliers else pd.DataFrame()
print("\n=== 이상치(급등) 후보 (최신 50행 모음) ===")
display(outliers_df if len(outliers_df) else pd.DataFrame({"info":["후보 없음(기준 미충족 또는 관측 부족)"]}))

# ============== 8) 공급-가격 상관 & 시차 상관 ==============
rows = []
if set(["반입량","금액"]).issubset(df.columns):
    for it in items:
        dfi = df[df[item_col]==it].sort_values(date_col).copy()
        if len(dfi) < 10: 
            continue
        same = dfi[["반입량","금액"]].corr().iloc[0,1]
        row = {"품목명": it, "동일일 상관(반입량-금액)": same}
        for lag in range(1, 8):
            dfi[f"반입량_lag{lag}"] = dfi["반입량"].shift(lag)
            row[f"lag{lag} 상관(반입량_lag{lag}-금액)"] = dfi[["금액", f"반입량_lag{lag}"]].corr().iloc[0,1]
        rows.append(row)

corr_df = pd.DataFrame(rows) if rows else pd.DataFrame()
print("\n=== 반입량-금액 상관/시차 상관(품목별) ===")
display(corr_df if len(corr_df) else pd.DataFrame({"info":["계산 불가(필요 컬럼/샘플 부족)"]}))

# (선택) 저장
if SAVE_DIR:
    na_table.to_csv(os.path.join(SAVE_DIR, "결측_전체.csv"), index=True, encoding="utf-8-sig")
    item_na_pct.to_csv(os.path.join(SAVE_DIR, "결측_품목별x칼럼.csv"), encoding="utf-8-sig")
    if len(outliers_df):
        outliers_df.to_csv(os.path.join(SAVE_DIR, "이상치_급등후보.csv"), index=False, encoding="utf-8-sig")
    if len(corr_df):
        corr_df.to_csv(os.path.join(SAVE_DIR, "상관_시차상관.csv"), index=False, encoding="utf-8-sig")
    print(f"\n[저장 완료] 폴더: {SAVE_DIR}")


ValueError: `거래일자`(또는 date 유사) 컬럼을 찾을 수 없습니다.